In [ ]:
!pip install elephant

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import torch

from google.colab import drive
import pickle

import plotly.express as plx
import matplotlib.pyplot as plt


import seaborn as sns

import gc

import scipy.signal as signal
from scipy.signal import welch, find_peaks

from neo import SpikeTrain
from elephant.conversion import BinnedSpikeTrain
import elephant.spike_train_correlation as elstc
import elephant.statistics as elstat
import quantities as pq
from elephant.statistics import isi

In [ ]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading training dataset 

**LOADING DATASETS**

In [ ]:
import pickle

# List of PCB indices
pcbs = [9, 14, 15, 16, 17, 18, 42, 48, 49]

# List to store DataFrames - for each PCB
df_list = []
# path to where you saved the feature dataset
path = "/content/drive/MyDrive/trainning_dataset"
# Loop through the PCB indices and load the corresponding pickle files
for pcb in pcbs:
    try:
        # Load the pickle file into a DataFrame
        file_path = os.path.join(path, f"pcb{pcb}.pkl")
        with open(file_path, "rb") as file:
            final_features = pickle.load(file)
            if pcb == 42 or pcb == 48 or pcb == 49:
                final_features = final_features.set_index(
                    ["day", "time_offset", "strain"]
                )
            df_list.append(final_features)
            print(f"Loaded PCB {pcb} successfully.")
    except FileNotFoundError:
        print(f"File for PCB {pcb} not found.")
    except Exception as e:
        print(f"Error loading PCB {pcb}: {e}")

# df_list contains all the features extracted for each pcb

Loaded PCB 9 successfully.
Loaded PCB 14 successfully.
Loaded PCB 15 successfully.
Loaded PCB 16 successfully.
Loaded PCB 17 successfully.
Loaded PCB 18 successfully.
Loaded PCB 42 successfully.
Loaded PCB 48 successfully.
Loaded PCB 49 successfully.


In [ ]:
def element_wise_average(column):
    """
    Parameters:
        column: for matching rows the column where we need to calculate the element-wise avergae
    Returns: element-wise average
    """
    stacked = np.stack(column.to_numpy())
    # Calculate the mean along rows, not taking into count the nan values
    return np.nanmean(stacked, axis=0).tolist()

In [ ]:
def row_element_wise_average(row, columns):
    """
    Parameters:
        row: the row which we are calculating the average for
        columns: the columns from which we need to calculate the average
    Returns: average across the columns in the row
    """
    stacked = np.stack([row[col] for col in columns])
    # Compute the element-wise mean
    return np.mean(stacked, axis=0).tolist()

In [1]:
def has_list_with_nan(element):
    """
    Parameters:
        element: element from dataframe, list or float
    Returns: Bool, check if list contains nan values
    """
    if isinstance(element, list):
        return any(
            pd.isna(x) for x in element
        )  # Check if any element in the list is NaN
    return False

**The section below processes the features from each PCB, getting rid of all the NaN values**

1. If there is a combination of (day,time_offset, strain) that matches the one of the element with NaN then we calculate the average across PCBs

2. If there are no other combinations of (day, time_offset,strain) that match from other PCBs then we calculate the average across the same PCB but amongs different MEA with data, because they are lists of features, we do element wise average across other MEAs

In [ ]:
match_columns = ["day", "time_offset", "strain"]

# Loop through each DataFrame and fill NaNs
for idx, df in enumerate(df_list):
    print(f"Processing PCB {idx+1}...")
    nan_rows = df[df.isna().any(axis=1)]
    # look for columns with NaN values (either float NaN or any NaN inside a list)
    columns_with_nan_in_lists = df.applymap(has_list_with_nan).any()
    columns_with_nan = columns_with_nan_in_lists[
        columns_with_nan_in_lists
    ].index.tolist()

    for row_index, row in nan_rows.iterrows():
        nan_columns = row[row.isna()].index.tolist()
        nan_columns.extend(columns_with_nan)
        first = True
        for ind, other_df in enumerate(df_list):
            # looking for PCBs with the same (day,time_offset,strain)
            if ind != idx:

                mask = (
                    (other_df.index.get_level_values("day") == row_index[0])
                    & (other_df.index.get_level_values("time_offset") == row_index[1])
                    & (other_df.index.get_level_values("strain") == row_index[2])
                )
                if first:
                    first = False
                    matching_rows = other_df.loc[mask]
                    matching_rows = matching_rows[nan_columns]
                else:
                    current = other_df.loc[mask]
                    current = current[nan_columns]
                    matching_rows = pd.concat([matching_rows, current], axis=0)
        # case 2
        if matching_rows.empty:
            # get the avergae of the surrounding MEA for MEA replacement and PSD for PSD replacement and Interactions for interactions
            for col in nan_columns:
                if col.startswith("MEA"):
                    # mea
                    other_mea = [
                        x
                        for x in df.columns
                        if x.startswith("MEA") and x not in nan_columns
                    ]
                    avg_value = row_element_wise_average(row, other_mea)
                    df.at[row_index, col] = avg_value
                elif col.startswith("PSD"):
                    # psd
                    other_psd = [
                        x
                        for x in df.columns
                        if x.startswith("PSD") and x not in nan_columns
                    ]
                    avg_value = row_element_wise_average(row, other_psd)
                    df.at[row_index, col] = avg_value
                else:
                    # interaction
                    other_int = [
                        x
                        for x in df.columns
                        if x.startswith("Interaction") and x not in nan_columns
                    ]
                    avg_value = row_element_wise_average(row, other_int)
                    df.at[row_index, col] = avg_value
        # case 1
        else:
            averages = {
                col: element_wise_average(matching_rows[col])
                for col in matching_rows.columns
            }
            for col in nan_columns:
                if not np.any(np.isnan(averages[col])):
                    df.at[row_index, col] = averages[col]
                else:
                    # Example Interactions where all interactions are nan
                    other_int = [
                        x
                        for x in df.columns
                        if x.startswith("Interaction") and x not in nan_columns
                    ]
                    avg_value = row_element_wise_average(row, other_int)
                    df.at[row_index, col] = avg_value

print("NaN values processed successfully.")

Processing PCB 1...
Processing PCB 2...
Processing PCB 3...
Processing PCB 4...
Processing PCB 5...
Processing PCB 6...
Processing PCB 7...
Processing PCB 8...


<ipython-input-128-f0d457038a42>:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  columns_with_nan_in_lists = df.applymap(has_list_with_nan).any()
<ipython-input-128-f0d457038a42>:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  columns_with_nan_in_lists = df.applymap(has_list_with_nan).any()
<ipython-input-128-f0d457038a42>:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  columns_with_nan_in_lists = df.applymap(has_list_with_nan).any()
<ipython-input-128-f0d457038a42>:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  columns_with_nan_in_lists = df.applymap(has_list_with_nan).any()
<ipython-input-128-f0d457038a42>:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  columns_with_nan_in_lists = df.applymap(has_list_with_nan).any()
<ipython-input-128-f0d457038a42>:13: FutureWarning: DataFrame.applymap

Processing PCB 9...
NaN values processed successfully.


In [ ]:
df_list[-1]

MEA1  \
day time_offset strain                                                      
1   0min        0.0     [1.30995920110016, 192.7524065314985, 1.267024...   
2   0min        0.0     [10.918426887350787, 23.804986855757974, 2.899...   
4   0min        0.0     [0.16259933021963616, 20.465872897887927, 0.96...   
7   0min        0.0     [0.3499000117936424, 88.00854966312755, 0.6601...   
9   0min        0.0     [0.1729094594805906, 32.129878875564835, 1.241...   
    10min       0.0     [1.806635358622668, 0.9898133635550284, 1.4715...   
    1h          0.0     [2.3762700002904023, 0.9743846926820419, 1.200...   
    2h          0.0     [3.2481203059198647, 0.4827270171126714, 1.194...   
10  0min        0.0     [0.7919468038570654, 12.554064983975929, 1.176...   
    10min       7.5     [0.18386397182035474, 34.014689030181756, 1.04...   
    1h          7.5     [2.379921504403657, 5.74737090321599, 1.211217...   
    2h          7.5     [4.242403396523161, 13.193067089518047, 1.1315...   
11  0min        7.5     [0.146704547608998, 9.012438646905624, 1.95349...   

                                                                     PSD1  \
day time_offset strain                                                      
1   0min        0.0     [-1.1405462424308969e-39, 2.8566506315285286e-...   
2   0min        0.0     [-1.2048258781108663e-37, 2.497412429553849e-3...   
4   0min        0.0     [-5.811042296430387e-39, 1.2097192028952048e-3...   
7   0min        0.0     [1.8392113387261172e-39, -3.1525243683371016e-...   
9   0min        0.0     [-1.4418760541403777e-39, 3.6519011591023956e-...   
    10min       0.0     [-1.0883773526485608e-38, 2.320527333042545e-3...   
    1h          0.0     [-2.927041326196673e-38, 6.1076210389472645e-3...   
    2h          0.0     [-1.590653710252958e-38, 3.358969150352845e-34...   
10  0min        0.0     [-1.5437716296736285e-39, 3.723196508352809e-3...   
    10min       7.5     [2.004476423392174e-39, -3.4609135380589816e-3...   
    1h          7.5     [-1.1384101656016221e-38, 2.3770021597652177e-...   
    2h          7.5     [-6.20127758756974e-39, 1.3408609163024982e-34...   
11  0min        7.5     [-1.261469372236317e-38, 2.68996925590045e-34,...   

                                                                     MEA2  \
day time_offset strain                                                      
1   0min        0.0     [1.1641854290494424, 12.384501945440306, 1.436...   
2   0min        0.0     [0.19540149742189908, 53.80203000224559, 0.850...   
4   0min        0.0     [0.6643589542527538, 28.715984393287727, 1.116...   
7   0min        0.0     [1.3991704584553633, 14.889025272242911, 1.073...   
9   0min        0.0     [0.2821538707747874, 38.8650243897466, 0.97098...   
    10min       0.0     [0.7590218970207435, 79.28780832879193, 0.8687...   
    1h          0.0     [1.7097631024416162, 22.252688731386858, 0.957...   
    2h          0.0     [2.9361775259589615, 82.2269892319733, 0.96233...   
10  0min        0.0     [0.8660508579201756, 35.91080316128227, 1.1653...   
    10min       7.5     [1.1903289711040055, 18.06018626113619, 1.2305...   
    1h          7.5     [2.1963030118514197, 24.06973580797404, 1.0825...   
    2h          7.5     [2.927340272306716, 35.0313845298792, 1.134213...   
11  0min        7.5     [0.4832873091072409, 9.067473216398426, 2.5544...   

                                                                     PSD2  \
day time_offset strain                                                      
1   0min        0.0     [3.356178462780562e-39, -6.594476728901217e-35...   
2   0min        0.0     [4.000805583265674e-39, -7.673274280490842e-35...   
4   0min        0.0     [-7.514373226963374e-39, 1.5598181663453119e-3...   
7   0min        0.0     [-9.75456956189694e-38, 2.0322376578694627e-33...   
9   0min        0.0     [-4.896915173999666e-39, 1.0951206393014804e-3...   
    10min       0.0     [-5.3754748504380086e-39, 1.1960645716

In [ ]:
def concat_data(df):
    """
    Parameters:
      df: dataframe with features for one PCB
    Returns:
      df: dataframe with each feature in an individual column
    """
    data = []
    index = []

    for header, row in df.iterrows():
        day = header[0]
        time_offset = header[1]
        strain = header[2]
        pcb_row = df.loc[day, time_offset, strain]

        # Flatten the data in pcb_row
        new_row = []
        for col_name, value in pcb_row.items():
            value = np.array(value)
            if not np.isnan(value).any():
                if col_name.startswith("MEA"):
                    mea_len = len(value)
                elif col_name.startswith("PSD"):
                    psd_len = len(value)
                new_row.extend(list(value))
            else:
                if col_name.startswith("MEA"):
                    new_row.extend([np.nan] * mea_len)
                elif col_name.startswith("PSD"):
                    new_row.extend([np.nan] * psd_len)

        data.append(new_row)
        index.append((day, time_offset, strain))

    # Construct the new DataFrame
    new_df = pd.DataFrame(
        data,
        index=pd.MultiIndex.from_tuples(index, names=["day", "time_offset", "strain"]),
    )

    return new_df

In [ ]:
df_strained_list = df_list[-3:]

In [ ]:
store_path = "/content/drive/MyDrive/data"
for idx, df in enumerate(df_strained_list):
    df = concat_data(df)  # separating each feature into individual columns
    store_path = os.path.join(store_path, f"data_strained_{idx}.pkl")
    df.to_pickle(store_path)

                             0          1         2          3          4    \
day time_offset strain                                                        
1   0min        0.0     0.624106  71.917299  1.969052  11.848490  14.442471   
2   0min        0.0     1.377777  98.514155  0.711785  11.713569  18.086366   
5   0min        0.0     1.660347  37.993136  0.986114  10.524614  14.940747   
6   0min        0.0     1.453041  60.086386  0.796751   5.417581  16.669587   
    10min       5.0     1.304653  47.345269  1.380794   6.521497  16.628831   
    1h          5.0     4.495904   1.811941  0.880507  12.075826  19.114785   
    2h          5.0     4.376994  70.615723  0.868934   3.974497  17.674250   
    10min       7.5     1.289625  36.121969  1.837080   4.417029  17.694718   
    1h          7.5     4.460764  30.703112  0.889857   9.269768  15.778724   
    2h          7.5     3.105497   5.085435  0.818294  13.548748  15.821928   
    10min       10.0    3.081869  18.517430  1.44926

In [ ]:
store_path = "/content/drive/MyDrive/data"
for idx, df in enumerate(df_list):
    df = concat_data(df)  # separating each feature into individual columns
    store_path = os.path.join(store_path, f"data_{idx}.pkl")
    df.to_pickle(store_path)  # save the fully processed datasets